In [59]:
import pandas as pd
import numpy as np

from rl_lib.swiss_round.environment import SwissRoundEnv
from rl_lib.swiss_round.agent import DQNAgent
from rl_lib.swiss_round.utils import probability_tables, check_probability

%reload_ext autoreload
%autoreload 2

# Environement

In [60]:
n_teams = 18
max_str = 16
threshold_ranks = [4,12]
agent_id = threshold_ranks[-1] +1
team_strengths = np.linspace(max_str,0,n_teams)
#team_strengths = [4 * 0.9 ** i for i in range(n_teams)]
mdp = 0.4

print(np.array(team_strengths).round(2))
wp, dp, lp = probability_tables(team_strengths=team_strengths, max_draw_probability=mdp)
pd.concat([wp.loc[[agent_id]].rename(index={agent_id:'Win'}),
           dp.loc[[agent_id]].rename(index={agent_id:'Draw'}),
           lp.loc[[agent_id]].rename(index={agent_id:'Loss'}),
           ]).T.round(2)

[16.   15.06 14.12 13.18 12.24 11.29 10.35  9.41  8.47  7.53  6.59  5.65
  4.71  3.76  2.82  1.88  0.94  0.  ]


,Win,Draw,Loss
0,0.00,0.00,1.00
1,0.00,0.00,1.00
2,0.00,0.00,1.00
3,0.00,0.00,1.00
4,0.00,0.00,1.00
5,0.00,0.00,1.00
6,0.00,0.00,1.00
7,0.00,0.00,1.00
8,0.01,0.00,0.99
9,0.02,0.01,0.97


In [61]:
n_rounds = 6
bonus_points = [30,20]
#Agent_id just below last threshold
n_baselines_simu = 2000
name = f'lin_{max_str}'
print(f"Valid probability set-up : {check_probability(team_strengths, mdp)}")

env = SwissRoundEnv(
    n_teams=n_teams,
    n_rounds=n_rounds,
    name=name,
    team_strengths=team_strengths,
    threshold_ranks=threshold_ranks,
    bonus_points=bonus_points,
    agent_id=agent_id,
    max_draw_probability=mdp
)

Valid probability set-up : True


#### Detailed tables

# Simulations

### Logs

In [62]:
cpdf = pd.read_csv('../logs/calmip_logs.csv', index_col=0)
locdf = pd.read_csv('../logs/exp_logs.csv', index_col=0)
ldf = pd.concat([cpdf,locdf]).reset_index(drop=True)

In [63]:
display_cols = ['exp_id', 'env_name','agent_id', 'n_episodes','train_epochs','lr','use_lr_scheduler','gradient_clipping','epsilon_end','epsilon_decay','activation', 'layers', 'dropout', 'avg_test_rewards',
       'std_test_rewards', 'avg_test_gambits', 'std_test_gambits']
query = (ldf['env_name'] == name) & (ldf['n_episodes']>=800) & (ldf['agent_id'] == agent_id)

In [64]:
ldf[query].sort_values(by='avg_test_rewards', ascending=False)[display_cols].head(20)

,exp_id,env_name,agent_id,n_episodes,train_epochs,lr,use_lr_scheduler,gradient_clipping,epsilon_end,epsilon_decay,activation,layers,dropout,avg_test_rewards,std_test_rewards,avg_test_gambits,std_test_gambits


### Baselines Simulations

In [65]:
simulation_wa = env.simulate_n_tournaments(n_baselines_simu, n_cores = 32,  policy = 'win_all', display_results=True)
simulation_lf = env.simulate_n_tournaments(n_baselines_simu,n_cores = 32, policy = 'lose_first',display_results=False)
baseline_reward_wa = simulation_wa.loc[agent_id,'Avg_Reward'] 
baseline_std_wa = simulation_wa.loc[agent_id,'Std_Reward'] 
baseline_reward_lf = simulation_lf.loc[agent_id,'Avg_Reward'] 
baseline_std_lf = simulation_lf.loc[agent_id,'Std_Reward'] 


Simulating tournaments with win_all policy: 100%|██████████| 2000/2000 [00:14<00:00, 136.64it/s]



Simulation Results (from 1958 tournaments):
Team | Strength | Avg Points | Avg Rank | Avg Reward | Std Reward | Top-4 % | Top-12 % | 
----------------------------------------------------------------------------------------
   0 |    16.00 |      16.44 |     1.50 |      66.13 |       3.86 |  98.98% | 100.00% | 
   1 |    15.06 |      14.56 |     2.39 |      62.44 |       8.79 |  92.95% | 100.00% | 
   2 |    14.12 |      13.06 |     3.39 |      57.16 |      13.08 |  80.34% | 100.00% | 
   3 |    13.18 |      11.94 |     4.57 |      48.07 |      16.12 |  53.83% |  99.90% | 
   4 |    12.24 |      11.23 |     5.44 |      42.10 |      15.63 |  36.41% |  99.74% | 
   5 |    11.29 |      10.65 |     6.32 |      36.33 |      13.14 |  19.46% |  99.23% | 
   6 |    10.35 |      10.01 |     7.35 |      32.28 |      10.05 |   8.84% |  98.11% | 
   7 |     9.41 |       9.63 |     8.07 |      30.25 |       9.31 |   5.31% |  95.10% | 
   8 |     8.47 |       9.06 |     9.16 |      27.90 |       8.5

Simulating tournaments with lose_first policy: 100%|██████████| 2000/2000 [00:15<00:00, 128.31it/s]


In [66]:
bldf = pd.concat([pd.DataFrame(simulation_wa.loc[agent_id]).T.rename({agent_id :'Win All'}),
                  pd.DataFrame(simulation_lf.loc[agent_id]).T.rename({agent_id :'Lose First'})])

print(f"Baselines average reward : WinAll = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}, "
      f"LoseFirst = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")
bldf.drop(columns = ['Team', 'Strength']).round(2)

Baselines average reward : WinAll = 12.6 ± 10.5, LoseFirst = 11.9 ± 10.3


,Avg_Points,Avg_Rank,Avg_Reward,Std_Reward,Top-4 %,Top-12 %
Win All,6.56,13.50,12.59,10.51,0.0,0.30
Lose First,6.43,13.74,11.90,10.34,0.0,0.27


# Appendix

### Environment probability tables

In [17]:
np.allclose((wp+dp+lp).values, np.ones(wp.shape))

True

In [6]:
wp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00,1.00,1.00,1.00,1.00,1.00
1,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00,1.00,1.00,1.00,1.00
2,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00,1.00,1.00,1.00
3,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00,1.00,1.00
4,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00,1.00
5,0.08,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99,1.00
6,0.05,0.08,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99,0.99
7,0.04,0.05,0.08,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98,0.99
8,0.02,0.04,0.05,0.08,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97,0.98
9,0.01,0.02,0.04,0.05,0.08,0.12,0.18,0.24,0.31,0.36,0.49,0.62,0.73,0.82,0.88,0.92,0.95,0.97


In [7]:
dp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00
3,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00
4,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00
5,0.04,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00
6,0.02,0.04,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00,0.00
7,0.01,0.02,0.04,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01,0.00
8,0.01,0.01,0.02,0.04,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01,0.01
9,0.01,0.01,0.01,0.02,0.04,0.06,0.09,0.13,0.20,0.29,0.20,0.13,0.09,0.06,0.04,0.02,0.01,0.01


In [8]:
lp.round(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00
1,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00
2,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.00
3,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00,0.00,0.00
4,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00,0.00
5,0.88,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01,0.00
6,0.92,0.88,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01,0.01
7,0.95,0.92,0.88,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01,0.01
8,0.97,0.95,0.92,0.88,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02,0.01
9,0.98,0.97,0.95,0.92,0.88,0.82,0.73,0.62,0.49,0.36,0.31,0.24,0.18,0.12,0.08,0.05,0.04,0.02


### RL Agent

In [67]:
print(f"Baselines average reward : WinAll = {baseline_reward_wa:.1f} ± {baseline_std_wa:.1f}, "
      f"LoseFirst = {baseline_reward_lf:.1f} ± {baseline_std_lf:.1f}")

Baselines average reward : WinAll = 12.6 ± 10.5, LoseFirst = 11.9 ± 10.3


In [14]:
agent = DQNAgent(env,
                 hidden_dims=[128,32],
                 dropout= 0.3,
                 lr = 0.0005,
                 buffer_size=10000,
                 epsilon_decay=0.9995,
                 activation='tanh',
                 train_epochs=8,
                 max_grad_norm=1,
                 use_lr_scheduler=True,
                 n_train_episodes=6000,
                 n_test_episodes=2000)
agent.train()
agent.evaluate()


--- Training in progress ---


KeyboardInterrupt: 

### Verbosed simulation

In [22]:
# Simulate tournament
final_standings = env.simulate_tournament(verbose= True)

print("\nFinal standings (team_id, points, opponent_average):")
for rank, (team_id, points, reward, opp_avg,strength) in enumerate(final_standings, 1):

    print(f"Rank {rank}: Team {team_id} - Strength {strength:.2f} - Points: {points} (reward {reward}) - Opponent Avg: {opp_avg:.2f}")


--- Simulating round n°1 ---
Game : Team 8 (points : 0, strength : 2.12) vs Team 2 (points : 0, strength : 3.53) : Draw
Game : Team 5 (points : 0, strength : 2.82) vs Team 1 (points : 0, strength : 3.76) : Team 1 wins
Game : Team 12 (points : 0, strength : 1.18) vs Team 11 (points : 0, strength : 1.41) : Team 11 wins
Game : Team 17 (points : 0, strength : 0.00) vs Team 9 (points : 0, strength : 1.88) : Team 9 wins
Game : Team 0 (points : 0, strength : 4.00) vs Team 10 (points : 0, strength : 1.65) : Team 10 wins
Game : Team 3 (points : 0, strength : 3.29) vs Team 6 (points : 0, strength : 2.59) : Draw
Game : Team 15 (points : 0, strength : 0.47) vs Team 14 (points : 0, strength : 0.71) : Team 14 wins
Game : Team 13 (points : 0, strength : 0.94) vs Team 7 (points : 0, strength : 2.35) : Team 7 wins
Game : Team 4 (points : 0, strength : 3.06) vs Team 16 (points : 0, strength : 0.24) : Team 4 wins
--- Simulating round n°2 ---
Game : Team 1 (points : 3, strength : 3.76) vs Team 4 (points :